<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Smoothma function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'> The Smoothma() function applies a smoothing function to a time series which results in a series that highlights the time series mean. For non-stationary time series with non-constant means, the smoothing function is used to create a result series. When the result series is subtracted from the original series, it removes the non-stationary mean behavior..</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    in_schema,
    TDSeries,
    Smoothma,
    Figure,
    plot,
    db_drop_view,
    remove_context
    )
import datetime
# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_Smoothma_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_AnomalyDetection_cloud');"
 # Takes about 50 seconds
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_AnomalyDetection_local');"
 # Takes about 2 minute 30 secs

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
Sensor_Data = DataFrame(in_schema('DEMO_AnomalyDetection', 'Sensor_Data'))
Sensor_Data

<p style = 'font-size:16px;font-family:Arial'>We get the above data from sensors. We focus on one plant (PLANT=1) and one robot (ROBOT_ID=41). The Partition_ID is the type of welding, ID is the WELDING_ID, X is time required for welding in ms and Y is the RESISTANCE. We create a view with the columns required to get data with proper column names.</p>

In [ ]:
%%capture
query = f"""
REPLACE VIEW DEMO_AnomalyDetection.V_dataset_01 AS
SELECT
    1 AS PLANT
,   {41} AS ROBOT_ID
,   CAST(A.PARTITION_ID AS BIGINT) AS WELDING_TYPE
,   CAST((DATE '{str(datetime.datetime.now()).split(' ')[0]}'  + FLOOR((WELDING_ID-700*WELDING_TYPE)/100))  AS DATE FORMAT 'YYYY-MM-DD') AS WELDING_DAY
,   CAST(A.ID AS BIGINT) AS WELDING_ID
,   CAST(A.X AS INTEGER) AS TIME_MS
,   A.Y AS RESISTANCE
FROM DEMO_AnomalyDetection.Sensor_Data A
"""
execute_sql(query)

In [ ]:
welding_dataset_new = DataFrame(in_schema('DEMO_AnomalyDetection', 'V_dataset_01'))
welding_dataset_new

In [ ]:
welding_dataset_new.shape

In [ ]:
id_curve = 854
single_welding = welding_dataset_new[welding_dataset_new.WELDING_ID == id_curve].sort('TIME_MS')
single_welding

In [ ]:
figure = Figure(width=1000, height=400, image_type="jpg",
                        heading="RESISTANCE")
plot = single_welding.plot(x=single_welding.TIME_MS, y=single_welding.RESISTANCE,
                    style='blue',xlabel='time in ms', ylabel='resistance ',figure=figure)
plot.show()

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. Smoothma</b>
<p style = 'font-size:16px;font-family:Arial'>Execute Smoothma() function to Perform exponential moving average.</p>

<p style = 'font-size:16px;font-family:Arial'>The first step is to convert the time series data into TDSeries, which is required for the input time series which are passed to the convolve function.</p>


In [ ]:
data_series_df= TDSeries(data=single_welding,
                             id="WELDING_ID",
                             row_index="TIME_MS",
                             row_index_style="SEQUENCE",
                             payload_field="RESISTANCE",
                             payload_content="REAL")

In [ ]:
# Example 1 :  Perform exponential moving average.
uaf_out = Smoothma(data=data_series_df,
                       ma='EXPONENTIAL',
                       lambda1=0.5)

In [ ]:
smooth_df = uaf_out.result
smooth_df

In [ ]:
figure = Figure(width=1000, height=400, image_type="jpg",
                        heading="RESISTANCE")
plot = smooth_df.plot(x=smooth_df.ROW_I, y=smooth_df.RESISTANCE,
                    style='blue',xlabel='time in ms', ylabel='resistance ',figure=figure)
plot.show()

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up intermediate tables.</p>

In [ ]:
db_drop_view(view_name='V_dataset_01', schema_name='DEMO_AnomalyDetection')

<p style = 'font-size:18px;font-family:Arial'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_AnomalyDetection');" 
#Takes 45 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;">

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>Smoothma function reference: <a href = 'https://docs.teradata.com/search/all?query=Smoothma&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>